# Personalized Restaurant Recommendation (Improved - No Data Leakage)

**Task.** Given a user `u` and an unseen restaurant `i`, predict a personalized recommendation score $\hat{r}_{u,i}$ (used for ranking). We evaluate rating prediction with **RMSE** and **MAE**.

**Key Improvements over `baseline.ipynb`:**
- **Fixed data leakage**: Removed use of current `review_text` which wouldn't be available at prediction time
- **Proper user features**: Extract user preferences from `history_reviews` (past reviews)
- **Sentiment analysis**: Added sentiment features from historical reviews

**Dataset.** Google Restaurants — dataset of restaurants from Google Local (Google Maps).

## 1. Data Loading and Preprocessing

In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from textblob import TextBlob
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error
from gensim.models import Word2Vec
import re
import random
import os


# Load data
with open("filter_all_t.json", "r") as f:
    for line in f:
        data = json.loads(line)

train_df = pd.DataFrame(data["train"])
val_df = pd.DataFrame(data.get("val", []))
test_df = pd.DataFrame(data.get("test", []))

# Remove pics column to save memory
train_df = train_df.drop(columns=["pics"], errors="ignore")
val_df = val_df.drop(columns=["pics"], errors="ignore")
test_df = test_df.drop(columns=["pics"], errors="ignore")

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

Train: 87013, Val: 10860, Test: 11015


In [2]:
# Compute training statistics (used for all models)
user_avg = train_df.groupby("user_id")["rating"].mean()
item_avg = train_df.groupby("business_id")["rating"].mean()
user_count = train_df.groupby("user_id").size()
item_count = train_df.groupby("business_id").size()
global_mean = train_df["rating"].mean()

print(f"Global mean rating: {global_mean:.3f}")
print(f"Unique users in train: {len(user_avg)}")
print(f"Unique items in train: {len(item_avg)}")

Global mean rating: 4.465
Unique users in train: 29596
Unique items in train: 27896


## 2. Feature Engineering

### Features:
- `user_avg`, `item_avg`: Average ratings from **training data only**
- `user_count`, `item_count`: Activity counts from **training data only**
- `history_reviews`: User's **past reviews** (available at prediction time)

In [3]:
def extract_history_features(history_reviews):
    """Extract features from user's historical reviews (NO LEAKAGE).
    
    Args:
        history_reviews: List of [review_id, review_text] pairs
    
    Returns:
        Dictionary of features extracted from historical reviews
    """
    # Parse historical review texts
    hist_texts = []
    if history_reviews:
        for h in history_reviews:
            if len(h) >= 2 and h[1]:
                hist_texts.append(h[1])
    
    if not hist_texts:
        return {
            'hist_count': 0,
            'hist_avg_len': 0,
            'hist_avg_polarity': 0,
            'hist_avg_subjectivity': 0,
            'hist_combined_text': ""
        }
    
    # Text statistics
    avg_len = np.mean([len(t) for t in hist_texts])
    
    # Sentiment analysis
    polarities = []
    subjectivities = []
    for text in hist_texts:
        try:
            blob = TextBlob(text)
            polarities.append(blob.sentiment.polarity)
            subjectivities.append(blob.sentiment.subjectivity)
        except:
            pass
    
    avg_polarity = np.mean(polarities) if polarities else 0
    avg_subjectivity = np.mean(subjectivities) if subjectivities else 0
    
    return {
        'hist_count': len(hist_texts),
        'hist_avg_len': avg_len,
        'hist_avg_polarity': avg_polarity,
        'hist_avg_subjectivity': avg_subjectivity,
        'hist_combined_text': " ".join(hist_texts)
    }

In [4]:
def add_features(df, user_avg, item_avg, user_count, item_count, global_mean):
    """Add legitimate features without data leakage."""
    df = df.copy()
    
    # User/item statistics from training (legitimate)
    df["user_avg"] = df["user_id"].map(user_avg).fillna(global_mean)
    df["item_avg"] = df["business_id"].map(item_avg).fillna(global_mean)
    df["user_count"] = df["user_id"].map(user_count).fillna(0)
    df["item_count"] = df["business_id"].map(item_count).fillna(0)
    
    # Extract history features with sentiment
    print("Extracting history features with sentiment analysis...")
    hist_features = df["history_reviews"].apply(extract_history_features)
    hist_df = pd.DataFrame(hist_features.tolist())
    
    df["hist_count"] = hist_df["hist_count"].values
    df["hist_avg_len"] = hist_df["hist_avg_len"].values
    df["hist_avg_polarity"] = hist_df["hist_avg_polarity"].values
    df["hist_avg_subjectivity"] = hist_df["hist_avg_subjectivity"].values
    df["hist_combined_text"] = hist_df["hist_combined_text"].values

    df["history_reviews_num"] = df['history_reviews'].apply(
    lambda x: len(x) if isinstance(x, list) else 0
    )
    df["low_hist_flag"] = (df["history_reviews_num"] <= 3).astype(int)
    
    return df

# Apply features to all splits
train_df = add_features(train_df, user_avg, item_avg, user_count, item_count, global_mean)
val_df = add_features(val_df, user_avg, item_avg, user_count, item_count, global_mean)
test_df = add_features(test_df, user_avg, item_avg, user_count, item_count, global_mean)

Extracting history features with sentiment analysis...
Extracting history features with sentiment analysis...
Extracting history features with sentiment analysis...


In [5]:
# Verify features
print("Features added:")
print(train_df[["user_avg", "item_avg", "hist_count", "hist_avg_polarity", "hist_avg_subjectivity", "history_reviews_num"]].describe())

Features added:
           user_avg      item_avg    hist_count  hist_avg_polarity  \
count  87013.000000  87013.000000  87013.000000       87013.000000   
mean       4.465252      4.465252      3.399825           0.334748   
std        0.576529      0.529461      4.476290           0.284386   
min        1.000000      1.000000      0.000000          -1.000000   
25%        4.100000      4.260870      1.000000           0.160000   
50%        4.500000      4.555556      2.000000           0.325000   
75%        5.000000      4.800000      4.000000           0.500000   
max        5.000000      5.000000     45.000000           1.000000   

       hist_avg_subjectivity  history_reviews_num  
count           87013.000000         87013.000000  
mean                0.574338             3.399837  
std                 0.218916             4.476282  
min                 0.000000             1.000000  
25%                 0.475000             1.000000  
50%                 0.596741             

## 3. Model 1: Linear Regression (Fixed - No TF-IDF on Current Review)

**Original (LEAKING):** Used TF-IDF on `review_text` (current review)

**Fixed:** Use only legitimate features:
- User/item averages and counts
- Historical review sentiment features

In [6]:
from sklearn.linear_model import Ridge

# Legitimate features only (NO current review_text or review_len)
feature_cols = [
    "user_avg", "item_avg", 
    "user_count", "item_count",
    "hist_count", "hist_avg_len",
    "hist_avg_polarity", "hist_avg_subjectivity"
]

X_train = train_df[feature_cols].values
X_val = val_df[feature_cols].values
X_test = test_df[feature_cols].values

y_train = train_df["rating"].values
y_val = val_df["rating"].values
y_test = test_df["rating"].values

# Use Ridge regression for stability
lr = Ridge(alpha=1.0)
lr.fit(X_train, y_train)

pred_val_lr = lr.predict(X_val)
pred_test_lr = lr.predict(X_test)

rmse_val_lr = np.sqrt(mean_squared_error(y_val, pred_val_lr))
rmse_test_lr = np.sqrt(mean_squared_error(y_test, pred_test_lr))
mae_val_lr = mean_absolute_error(y_val, pred_val_lr)
mae_test_lr = mean_absolute_error(y_test, pred_test_lr)

print("=" * 50)
print("Model 1: Ridge Regression (Fixed - No Leakage)")
print("=" * 50)
print(f"Val  RMSE: {rmse_val_lr:.4f}, MAE: {mae_val_lr:.4f}")
print(f"Test RMSE: {rmse_test_lr:.4f}, MAE: {mae_test_lr:.4f}")

Model 1: Ridge Regression (Fixed - No Leakage)
Val  RMSE: 1.2164, MAE: 0.8226
Test RMSE: 1.2473, MAE: 0.8320


In [7]:
# Feature importance
print("\nFeature Coefficients:")
for name, coef in zip(feature_cols, lr.coef_):
    print(f"  {name}: {coef:.4f}")


Feature Coefficients:
  user_avg: 0.7572
  item_avg: 0.6451
  user_count: -0.1470
  item_count: -0.0000
  hist_count: 0.1459
  hist_avg_len: 0.0001
  hist_avg_polarity: -0.1562
  hist_avg_subjectivity: 0.0252


## 4. Model 2: SVD + SBERT + MLP

In [8]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

user_map = {u: i for i, u in enumerate(train_df['user_id'].unique())}
item_map = {b: i for i, b in enumerate(train_df['business_id'].unique())}

rows = train_df['user_id'].map(user_map).values
cols = train_df['business_id'].map(item_map).values
data = train_df['rating'].values

# Sparse matrix
R_sparse = csr_matrix((data, (rows, cols)),
                      shape=(len(user_map), len(item_map)))

print("Sparse shape =", R_sparse.shape)
print("Sparsity =", 1 - R_sparse.count_nonzero() / (R_sparse.shape[0] * R_sparse.shape[1]))

# Fit SVD
n_factors = 20
svd = TruncatedSVD(n_components=n_factors, random_state=42)
U = svd.fit_transform(R_sparse)
V = svd.components_.T

print("SVD done")
print("Explained variance ratio:", svd.explained_variance_ratio_.sum())

Sparse shape = (29596, 27896)
Sparsity = 0.9998946076254966
SVD done
Explained variance ratio: 0.029474984686464912


In [9]:
user_means = train_df.groupby('user_id')['rating'].mean()
item_means = train_df.groupby('business_id')['rating'].mean()
global_mean = train_df['rating'].mean()

def svd_predict(df):
    """Safe SVD prediction using dot products (no large R_hat matrix)."""
    
    preds = []
    
    for _, row in df.iterrows():
        uid, bid = row['user_id'], row['business_id']
        
        has_user = uid in user_map
        has_item = bid in item_map
        
        if has_user and has_item:
            u_idx = user_map[uid]
            i_idx = item_map[bid]
            pred = np.dot(U[u_idx], V[i_idx])
        
        elif has_user:
            pred = user_means.get(uid, global_mean)
        
        elif has_item:
            pred = item_means.get(bid, global_mean)
        
        else:
            pred = global_mean
        
        preds.append(pred)
    
    return np.array(preds)

pred_val_svd = svd_predict(val_df)
pred_test_svd = svd_predict(test_df)

rmse_val_svd = np.sqrt(mean_squared_error(y_val, pred_val_svd))
rmse_test_svd = np.sqrt(mean_squared_error(y_test, pred_test_svd))
mae_val_svd = mean_absolute_error(y_val, pred_val_svd)
mae_test_svd = mean_absolute_error(y_test, pred_test_svd)

print("=" * 50)
print("Model 2: SVD (Already Leak-Free)")
print("=" * 50)
print(f"Val  RMSE: {rmse_val_svd:.4f}, MAE: {mae_val_svd:.4f}")
print(f"Test RMSE: {rmse_test_svd:.4f}, MAE: {mae_test_svd:.4f}")

Model 2: SVD (Already Leak-Free)
Val  RMSE: 0.9448, MAE: 0.6793
Test RMSE: 0.9123, MAE: 0.6630


In [10]:
from sentence_transformers import SentenceTransformer
import xgboost as xgb

# Load model
st_model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_dim = 128

print(f"Loaded SentenceTransformer with embedding dim: {embedding_dim}")

C:\Users\mio\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded SentenceTransformer with embedding dim: 128


In [11]:
def build_user_embedding_from_history(history_reviews, model):
    """Build user embedding from history_reviews ONLY (no leakage)."""
    if not history_reviews:
        return None  # let fallback handle it
    hist_texts = []
    if history_reviews:
        for h in history_reviews:
            if len(h) >= 2 and h[1]:
                hist_texts.append(h[1])
    
    if not hist_texts:
        return None
    
    lengths = np.array([len(t) for t in hist_texts])
    weights = lengths / lengths.sum()

    embs = model.encode(hist_texts, convert_to_tensor=True)
    user_emb = (embs * torch.tensor(weights, device=embs.device).unsqueeze(1)).sum(dim=0)
    return user_emb.cpu().numpy()


def build_item_embeddings(train_df, model, batch_size=16):
    """
    Build item embeddings for each business_id.
    Efficient, memory-safe, avoids SBERT bottlenecks.
    """
    item_emb_dict = {}

    # Pre-group but do NOT materialize whole groups at once
    grouped = train_df.groupby('business_id')['review_text']

    for bid, texts in tqdm(grouped, desc="Building item embeddings"):
        texts = [t for t in texts.dropna().tolist() if isinstance(t, str)]

        if len(texts) == 0:
            continue

        all_embs = []

        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            batch_embs = model.encode(
                batch,
                show_progress_bar=False,
                convert_to_tensor=False
            )
            all_embs.append(batch_embs)

        all_embs = np.vstack(all_embs)
        avg_emb = all_embs.mean(axis=0)
        item_emb_dict[bid] = avg_emb.tolist()

    return item_emb_dict

In [12]:
from sklearn.decomposition import PCA
# Build item embeddings from training data
print("Building item embeddings from training data...")
item_emb_dict = build_item_embeddings(train_df, st_model)
print(f"Built embeddings for {len(item_emb_dict)} items")

# Compute global embedding for cold-start fallback
global_item_emb = np.mean(list(item_emb_dict.values()), axis=0)

Building item embeddings from training data...


Building item embeddings: 100%|██████████| 27896/27896 [03:12<00:00, 144.66it/s]


Built embeddings for 27896 items


In [13]:
def safe_vector(x, dim):
    """Ensure x is a numpy vector of fixed dimension."""
    x = np.array(x, dtype=float).flatten()
    if len(x) != dim:
        return np.zeros(dim)
    return x


def build_hybrid_features_for_model(
    df,
    st_model,
    item_emb_dict,
    global_item_emb,
    svd_user_emb,
    svd_item_emb,
    embedding_dim=384,
    svd_dim=50
):
    X_list = []

    global_user_emb = np.zeros(embedding_dim)
    zero_svd_user = np.zeros(svd_dim)
    zero_svd_item = np.zeros(svd_dim)

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Building Hybrid Features"):

        # ---------- 1. RAW user embedding ----------
        u_emb = build_user_embedding_from_history(row["history_reviews"], st_model)
        u_emb = safe_vector(u_emb if u_emb is not None else global_user_emb, embedding_dim)

        # ---------- 2. RAW item embedding ----------
        i_emb = item_emb_dict.get(row["business_id"], global_item_emb)
        i_emb = safe_vector(i_emb, embedding_dim)

        # ---------- 3. SVD embeddings ----------
        u_svd = safe_vector(svd_user_emb.get(row['user_id'], zero_svd_user), svd_dim)
        i_svd = safe_vector(svd_item_emb.get(row['business_id'], zero_svd_item), svd_dim)

        # ---------- 4. numeric features ----------
        history_num = float(row.get("history_reviews_num", 0) or 0)
        low_hist_flag = 1 if history_num <= 3 else 0
        hist_pol = float(row.get("hist_avg_polarity", 0) or 0)
        hist_subj = float(row.get("hist_avg_subjectivity", 0) or 0)
        user_avg = float(row.get("user_avg", 0) or 0)
        item_avg = float(row.get("item_avg", 0) or 0)

        # ---------- 5. concat ----------
        feat = np.concatenate([
            u_emb,
            i_emb,
            u_svd,
            i_svd,
            [user_avg, item_avg, hist_pol, hist_subj, history_num, low_hist_flag]
        ])
        X_list.append(feat)

    return np.vstack(X_list)

In [14]:
svd_user_emb = {uid: U[user_map[uid]] for uid in user_map}
svd_item_emb = {bid: V[item_map[bid]] for bid in item_map}

In [15]:
X_train = build_hybrid_features_for_model(
    train_df,
    st_model,
    item_emb_dict,
    global_item_emb,
    svd_user_emb,
    svd_item_emb
)

Building Hybrid Features: 100%|██████████| 87013/87013 [10:29<00:00, 138.26it/s]


In [16]:
X_val = build_hybrid_features_for_model(
    val_df, st_model, item_emb_dict, global_item_emb,
    svd_user_emb, svd_item_emb
)

Building Hybrid Features: 100%|██████████| 10860/10860 [01:53<00:00, 95.78it/s] 


In [17]:
X_test = build_hybrid_features_for_model(
    test_df, st_model, item_emb_dict, global_item_emb,
    svd_user_emb, svd_item_emb
)

Building Hybrid Features: 100%|██████████| 11015/11015 [01:27<00:00, 125.97it/s]


In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

class RatingDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze(1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

train_ds = RatingDataset(X_train_scaled, y_train)
val_ds = RatingDataset(X_val_scaled, y_val)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=256, shuffle=False)

In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

model = MLP(input_dim=X_train_scaled.shape[1])
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
criterion = nn.MSELoss()

best_val = float('inf')
best_state = None
patience = 5
wait = 0

for epoch in range(50):
    # ---- train ----
    model.train()
    train_loss = 0.0
    for Xb, yb in train_loader:
        pred = model(Xb)
        loss = criterion(pred, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * len(Xb)
    train_loss /= len(train_ds)

    # ---- val ----
    model.eval()
    val_loss = 0.0
    val_preds = []
    val_trues = []
    with torch.no_grad():
        for Xb, yb in val_loader:
            pred = model(Xb)
            loss = criterion(pred, yb)
            val_loss += loss.item() * len(Xb)

            val_preds.append(pred.numpy())
            val_trues.append(yb.numpy())

    val_loss /= len(val_ds)
    val_preds = np.concatenate(val_preds)
    val_trues = np.concatenate(val_trues)
    val_rmse = np.sqrt(((val_preds - val_trues) ** 2).mean())
    val_mae  = np.abs(val_preds - val_trues).mean()

    print(f"Epoch {epoch:02d} | train MSE={train_loss:.4f} | "
          f"val MSE={val_loss:.4f} | val RMSE={val_rmse:.4f} | val MAE={val_mae:.4f}")

    if val_loss < best_val - 1e-4:
        best_val = val_loss
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping!")
            break

if best_state is not None:
    model.load_state_dict(best_state)

Epoch 00 | train MSE=0.9641 | val MSE=1.1342 | val RMSE=1.0650 | val MAE=0.8389
Epoch 01 | train MSE=0.5397 | val MSE=1.0040 | val RMSE=1.0020 | val MAE=0.8041
Epoch 02 | train MSE=0.4472 | val MSE=0.9625 | val RMSE=0.9811 | val MAE=0.8089
Epoch 03 | train MSE=0.3992 | val MSE=0.8976 | val RMSE=0.9474 | val MAE=0.7550
Epoch 04 | train MSE=0.3602 | val MSE=0.8057 | val RMSE=0.8976 | val MAE=0.6995
Epoch 05 | train MSE=0.3361 | val MSE=0.8218 | val RMSE=0.9065 | val MAE=0.6880
Epoch 06 | train MSE=0.3156 | val MSE=0.8352 | val RMSE=0.9139 | val MAE=0.6696
Epoch 07 | train MSE=0.3055 | val MSE=0.8126 | val RMSE=0.9014 | val MAE=0.6965
Epoch 08 | train MSE=0.2897 | val MSE=0.8029 | val RMSE=0.8961 | val MAE=0.6930
Epoch 09 | train MSE=0.2766 | val MSE=0.8336 | val RMSE=0.9130 | val MAE=0.6881
Epoch 10 | train MSE=0.2688 | val MSE=0.8434 | val RMSE=0.9184 | val MAE=0.6699
Epoch 11 | train MSE=0.2595 | val MSE=0.8117 | val RMSE=0.9010 | val MAE=0.6998
Epoch 12 | train MSE=0.2524 | val MSE=0.

In [20]:
model.eval()
X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32)
with torch.no_grad():
    pred_test = model(X_test_t).numpy()
mae_test = mean_absolute_error(y_test, pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, pred_test))

print(f"Test MAE:  {mae_test:.4f}")
print(f"Test RMSE: {rmse_test:.4f}")

Test MAE:  0.6770
Test RMSE: 0.8655


## 5. Model 3: Word2Vec + CNN 

In [21]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error
from gensim.models import Word2Vec
import re
from tqdm import tqdm
import random
import os

# ---------------------------
# Reproducibility
# ---------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# ---------------------------
# 1) Load data
# ---------------------------
print("Loading data...")
with open("filter_all_t.json", "r") as f:
    data = json.load(f)

train_df = pd.DataFrame(data["train"])
val_df   = pd.DataFrame(data.get("val", []))
test_df  = pd.DataFrame(data.get("test", []))

# Keep only the columns we need
train_df = train_df[['user_id', 'business_id', 'rating', 'history_reviews']]
val_df   = val_df[['user_id', 'business_id', 'rating', 'history_reviews']]
test_df  = test_df[['user_id', 'business_id', 'rating', 'history_reviews']]

print(f"Train size: {len(train_df)}, Val size: {len(val_df)}, Test size: {len(test_df)}")

# ---------------------------
# 2) Preprocessing helpers
# ---------------------------
def preprocess_text(text):
    """Fast preprocessing: lowercase, remove non-alpha, split."""
    if not isinstance(text, str):
        return []
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    tokens = text.split()
    return tokens

def extract_history_text(history_reviews, max_reviews=5):
    """Extract text from history reviews (could be list, array, or string)."""
    # Handle None or NaN
    if history_reviews is None or (isinstance(history_reviews, float) and pd.isna(history_reviews)):
        return ''
    
    # Handle empty string
    if isinstance(history_reviews, str) and history_reviews.strip() == '':
        return ''
    
    # Handle list or array
    if isinstance(history_reviews, (list, np.ndarray)):
        if len(history_reviews) == 0:
            return ''
        # Take the most recent reviews
        recent = history_reviews[-max_reviews:] if len(history_reviews) > max_reviews else history_reviews
        return ' '.join([str(r) for r in recent if r and str(r).strip()])
    
    # Handle string or other types
    return str(history_reviews)

# ---------------------------
# 3) Train Word2Vec on TRAIN history reviews only
# ---------------------------
print("Preparing Word2Vec training corpus from TRAIN history reviews...")
train_history_texts = []
for history in train_df['history_reviews']:
    history_text = extract_history_text(history)
    if history_text:
        train_history_texts.append(history_text)

sentences = [preprocess_text(t) for t in train_history_texts]
sentences = [s for s in sentences if len(s) > 3]  # remove very short sequences

print(f"Training Word2Vec on {len(sentences)} history review sequences...")
word2vec = Word2Vec(
    sentences=sentences,
    vector_size=100,
    window=3,
    min_count=5,
    workers=1,  # Single worker for reproducibility
    sg=1,
    epochs=8,
    seed=SEED
)

# Build word->idx mapping (PAD=0, UNK=1)
word_to_idx = {'<PAD>': 0, '<UNK>': 1}
for i, word in enumerate(word2vec.wv.key_to_index.keys(), start=2):
    word_to_idx[word] = i
vocab_size = len(word_to_idx)
print(f"Vocab size (with PAD/UNK): {vocab_size}")

# ---------------------------
# 4) Global user/item mappings from TRAIN
# ---------------------------
train_users = train_df['user_id'].unique()
train_items = train_df['business_id'].unique()

user_to_idx_global = {uid: i for i, uid in enumerate(train_users)}
item_to_idx_global = {bid: i for i, bid in enumerate(train_items)}

UNK_USER = len(user_to_idx_global)    # index for unseen users
UNK_ITEM = len(item_to_idx_global)    # index for unseen items

# Sizes for embeddings (add 1 for UNK)
NUM_USERS = len(user_to_idx_global) + 1
NUM_ITEMS = len(item_to_idx_global) + 1

print(f"Num train users: {len(user_to_idx_global)}, num train items: {len(item_to_idx_global)}")
print(f"NUM_USERS (with UNK): {NUM_USERS}, NUM_ITEMS (with UNK): {NUM_ITEMS}")

# ---------------------------
# 5) Dataset using HISTORY reviews
# ---------------------------
class RecommendationDataset(Dataset):
    def __init__(self, df, word_to_idx, max_length=200, max_history_reviews=5):
        self.df = df.reset_index(drop=True)
        self.word_to_idx = word_to_idx
        self.max_length = max_length
        self.max_history_reviews = max_history_reviews

        # Use global mappings
        self.user_to_idx = user_to_idx_global
        self.item_to_idx = item_to_idx_global

        # Precompute token indices from HISTORY reviews
        self.sequences = []
        self.history_lengths = []
        
        for history in self.df['history_reviews']:
            history_text = extract_history_text(history, max_reviews=max_history_reviews)
            tokens = preprocess_text(history_text)
            self.history_lengths.append(min(len(tokens), max_length))

            tokens = tokens[:max_length]
            indices = [word_to_idx.get(tok, 1) for tok in tokens]  # UNK=1
            if len(indices) < max_length:
                indices = indices + [0] * (max_length - len(indices))
            self.sequences.append(indices)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        u = self.user_to_idx.get(row['user_id'], UNK_USER)
        i = self.item_to_idx.get(row['business_id'], UNK_ITEM)

        return {
            'user_idx': torch.tensor(u, dtype=torch.long),
            'item_idx': torch.tensor(i, dtype=torch.long),
            'history_seq': torch.tensor(self.sequences[idx], dtype=torch.long),
            'history_length': torch.tensor(self.history_lengths[idx], dtype=torch.float32),
            'rating': torch.tensor(row['rating'], dtype=torch.float32)
        }


Loading data...
Train size: 87013, Val size: 10860, Test size: 11015
Preparing Word2Vec training corpus from TRAIN history reviews...
Training Word2Vec on 87013 history review sequences...
Vocab size (with PAD/UNK): 16186
Num train users: 29596, num train items: 27896
NUM_USERS (with UNK): 29597, NUM_ITEMS (with UNK): 27897


In [22]:
# ---------------------------
# 6) Model: Predict based on user history + user/item embeddings
# ---------------------------
import torch.nn.functional as F

class HistoryBasedRecommender(nn.Module):
    def __init__(self, num_users, num_items, vocab_size, embedding_dim=100):
        super().__init__()
        
        # User & item embeddings
        self.user_emb = nn.Embedding(num_users, 32)
        self.item_emb = nn.Embedding(num_items, 32)
        
        # Word embeddings for history reviews
        self.word_emb = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self._init_word_embeddings(word2vec, word_to_idx)
        
        # 1D CNN to encode user's history
        self.conv = nn.Conv1d(in_channels=embedding_dim, out_channels=128, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.dropout_text = nn.Dropout(0.2)
        
        # Predictor: combines user, item, and user history
        self.predictor = nn.Sequential(
            nn.Linear(32 + 32 + 128 + 1, 128),  # user + item + history + length
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    
    def _init_word_embeddings(self, word2vec, word_to_idx):
        """Initialize word embeddings with Word2Vec."""
        embedding_matrix = np.zeros((len(word_to_idx), 100))
        for word, idx in word_to_idx.items():
            if word in word2vec.wv:
                embedding_matrix[idx] = word2vec.wv[word]
            elif word == '<UNK>':
                embedding_matrix[idx] = np.random.normal(scale=0.1, size=(100,))
        self.word_emb.weight.data.copy_(torch.from_numpy(embedding_matrix))
        print(f"Initialized {np.sum(np.any(embedding_matrix != 0, axis=1))} words from Word2Vec")
    
    def forward(self, user_idx, item_idx, history_seq, history_length):
        # User & item embeddings
        user_emb = self.user_emb(user_idx)  # [batch, 32]
        item_emb = self.item_emb(item_idx)  # [batch, 32]
        
        # History text embeddings (CNN)
        word_embs = self.word_emb(history_seq)      # [batch, seq_len, embed_dim]
        x = word_embs.transpose(1, 2)               # [batch, embed_dim, seq_len]
        x = F.relu(self.conv(x))
        x = self.pool(x).squeeze(-1)                # [batch, 128]
        x = self.dropout_text(x)
        
        # Normalize history length
        history_length_norm = (history_length / 200.0).unsqueeze(1)  # [batch, 1]
        
        # Combine all features
        combined = torch.cat([user_emb, item_emb, x, history_length_norm], dim=1)
        rating = self.predictor(combined).squeeze(-1)
        return rating


In [23]:
# ---------------------------
# 7) Training & evaluation utilities
# ---------------------------
def evaluate_model(model, loader, device):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for batch in loader:
            user_idx = batch['user_idx'].to(device)
            item_idx = batch['item_idx'].to(device)
            history_seq = batch['history_seq'].to(device)
            history_length = batch['history_length'].to(device)
            rating = batch['rating'].to(device)

            out = model(user_idx, item_idx, history_seq, history_length)
            preds.extend(out.cpu().numpy())
            targets.extend(rating.cpu().numpy())

    rmse = np.sqrt(mean_squared_error(targets, preds))
    mae = mean_absolute_error(targets, preds)
    return rmse, mae, np.array(preds), np.array(targets)

In [24]:
# ---------------------------
# 8) Training loop
# ---------------------------
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def train_recommender(num_epochs=10, batch_size=128, lr=1e-3, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("Device:", device)

    train_dataset = RecommendationDataset(train_df, word_to_idx, max_length=200)
    val_dataset   = RecommendationDataset(val_df, word_to_idx, max_length=200)
    test_dataset  = RecommendationDataset(test_df, word_to_idx, max_length=200)

    g = torch.Generator()
    g.manual_seed(SEED)

    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=0,
        worker_init_fn=seed_worker,
        generator=g
    )
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    model = HistoryBasedRecommender(
        num_users=NUM_USERS, 
        num_items=NUM_ITEMS, 
        vocab_size=vocab_size
    ).to(device)
    
    print("Model params:", sum(p.numel() for p in model.parameters()))

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    best_val_rmse = float('inf')
    patience = 3
    patience_counter = 3

    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0.0
        train_preds, train_targets = [], []

        for batch in train_loader:
            user_idx = batch['user_idx'].to(device)
            item_idx = batch['item_idx'].to(device)
            history_seq = batch['history_seq'].to(device)
            history_length = batch['history_length'].to(device)
            rating = batch['rating'].to(device)

            optimizer.zero_grad()
            out = model(user_idx, item_idx, history_seq, history_length)
            loss = criterion(out, rating)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            train_preds.extend(out.detach().cpu().numpy())
            train_targets.extend(rating.cpu().numpy())

        train_rmse = np.sqrt(mean_squared_error(train_targets, train_preds))
        train_mae  = mean_absolute_error(train_targets, train_preds)

        val_rmse, val_mae, _, _ = evaluate_model(model, val_loader, device)

        print(f"\nEpoch {epoch}/{num_epochs}:")
        print(f"  Train RMSE: {train_rmse:.4f}, MAE: {train_mae:.4f}")
        print(f"  Val   RMSE: {val_rmse:.4f}, MAE: {val_mae:.4f}")

        # Early stopping
        if val_rmse < best_val_rmse:
            best_val_rmse = val_rmse
            best_model_state = model.state_dict().copy()
            print(" New best model")
            patience_counter = patience
        else:
            patience_counter -= 1
            print(f"  Patience left: {patience_counter}")
            if patience_counter <= 0:
                print("Early stopping triggered.")
                break

    # Test with best model
    print("\n" + "="*50)
    print("Evaluating best model on test set...")
    model.load_state_dict(best_model_state)
    test_rmse, test_mae, test_preds, test_targets = evaluate_model(model, test_loader, device)

    return test_rmse, test_mae


In [25]:
# ---------------------------
# 9) Main
# ---------------------------
if __name__ == "__main__":
    print("\n" + "="*50)
    print("TRAINING HISTORY-BASED RECOMMENDER")
    print("="*50)
    
    test_rmse, test_mae = train_recommender(num_epochs=10, batch_size=128, lr=1e-3)
    
    print("\n" + "="*50)
    print("FINAL TEST RESULTS")
    print("="*50)
    print(f"Test RMSE: {test_rmse:.4f}")
    print(f"Test MAE:  {test_mae:.4f}")
    print("="*50)


TRAINING HISTORY-BASED RECOMMENDER
Device: cuda
Initialized 16185 words from Word2Vec
Model params: 3530089

Epoch 1/10:
  Train RMSE: 1.1121, MAE: 0.8126
  Val   RMSE: 0.9160, MAE: 0.7693
 New best model

Epoch 2/10:
  Train RMSE: 0.9014, MAE: 0.6913
  Val   RMSE: 0.8593, MAE: 0.7040
 New best model

Epoch 3/10:
  Train RMSE: 0.8782, MAE: 0.6713
  Val   RMSE: 0.8922, MAE: 0.7433
  Patience left: 2

Epoch 4/10:
  Train RMSE: 0.8524, MAE: 0.6495
  Val   RMSE: 1.0096, MAE: 0.8730
  Patience left: 1

Epoch 5/10:
  Train RMSE: 0.8184, MAE: 0.6200
  Val   RMSE: 0.9382, MAE: 0.7932
  Patience left: 0
Early stopping triggered.

Evaluating best model on test set...

FINAL TEST RESULTS
Test RMSE: 0.9281
Test MAE:  0.7841
